In [1]:
# import libraries here; add more as necessary
from collections import namedtuple
import pickle
import math
import itertools
import importlib
import os
from time import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier

import azdias_helpers.clean_df as clean_df
import azdias_helpers.verbosity as v
import azdias_helpers.pickler as pickler


# magic word for producing visualizations in notebook
%matplotlib inline

pd.set_option('display.max_rows', None)

In [2]:
azdias = pd.read_csv('./Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('./Udacity_CUSTOMERS_052018.csv', sep=';')

C:\Users\Olive\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
importlib.reload(clean_df)

<module 'azdias_helpers.clean_df' from 'C:\\Users\\Olive\\Documents\\PortfolioProjects\\arvato-udacity-customersegmentation\\azdias_helpers\\clean_df.py'>

In [13]:
azdias_clean = clean_df.etl_pipeline(azdias,verbosity=v.MED)

Running etl_pipeline...
Running unknown_to_nan...
Finished unknown_to_nan...
Running onehot_encode_df...
Finished running onehot_encode_df...
Running remove_string_values...
Finished running remove_string_values.
Running binarize_nan...
Finishing binarize_nan.
Running standardise_binary_features...
Finished running standardise_binary_features.
Running bin_features_with_tails...
Finished running bin_features_with_tails
Running drop_sparse_samples.
0% of rows dropped: 0 out of 891,221
Finished running drop_sparse_samples.
Finished running etl_pipeline.

df shape before:(891221, 366), after:(891221, 99)


In [ ]:
customers_clean = clean_df.etl_pipeline(customers,verbosity=v.MED)

Running etl_pipeline...
Running unknown_to_nan...
Finished unknown_to_nan...
Running onehot_encode_df...
Finished running onehot_encode_df...
Running remove_string_values...
Finished running remove_string_values.
Running binarize_nan...
